---
# Primeiro Passo - configuração do ambiente
---

Tutorial: http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/get-set-up-for-amazon-ec2.html

## Criação de um usuário IAM (Instance and Access Management)

A Amazon não recomenda usar o login principal para esta finalidade, acredito que por razões de segurança da conta.

É recomendado criar usuários com poderes administrativos, que terão acesso a criação e uso de instâncias *dentro da conta principal*.

* Acessar: https://console.aws.amazon.com/iam/;
* **Users** -> **Add User**;
* Escolher username;
* Marcar a caixa "AWS Management Console access";
* Definir senha;
* Criar grupo e adicionar a adicionar a política: "AdministratorAccess", encontrada pela filtragem (ou adicionar ao grupo **rnpi**);
* Finalizar criação;
* Página de login será: https://your_aws_account_id.signin.aws.amazon.com/console/, onde your_aws_account_id é o ID da conta AWS.

## Criar um Key Pair

Para acessar a instância é necessário ter o arquivo da chave (.pem) salvo na máquina local. O único momento em que você consegue baixar este arquivo é na hora da criação do mesmo.

* Acessar: https://console.aws.amazon.com/ec2/;
* **Key Pairs** -> **Create Key Pair**;
* Definir um nome para o mesmo;
* Baixar o arquivo *.pem*;
* No terminal local digitar: ``chmod 400 <your_user_name-key-pair-region_name>.pem`` para que somente o seu usuário tenha permissão de acesso.

## Utilizar VPC padrão

Não sei bem o que faz, mas não parece ser necessário mexer nisso.

## Criar Security Group

Pelo que entendi é necessário liberar acesso SSH para cada IP que vá tentar conectar-se à instância.

Além disso adicionam-se algumas portas padrão e também adicionei a porta para acesso do Jupyter (que rodará através do Docker)

* Acessar: https://console.aws.amazon.com/ec2/;
* **Security Groups** -> **Add Security Groups**;
* Escolher um nome para o mesmo;
* Selecionar **Add Rule** e adicionar as seguintes regras:
    * Type: HTTP, Source: Anywhere (0.0.0.0/0)
    * Type: HTTPS, Source: Anywhere (0.0.0.0/0)
    * Type: SSH, Source: My IP (...) - *não recomendado liberar para todos os IPs*
    * Type: Custom TCP Rule, Port: ``<porta_jupyter>``, Source: Anywhere - *adicionado por mim*

---
# Segundo Passo - criação de uma instância
---

Tutorial: http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/EC2_GetStarted.html

* Acessar: https://console.aws.amazon.com/ec2/;
* Verificar se a região desejada já está selecionada (a princípio todas as máquinas estão sendo criadas em *US West (Oregon)*;
* Selecionar **Launch Instance**;
* Escolher a AMI (Amazon Machine Instance) - usei a padrão da Amazon (Amazon Linux AMI 2016.09.1 (HVM), SSD Volume Type - ami-1e299d7e);
* Escolher o tipo de instância (g2.2xlarge para uso moderado de GPU);
* Selecionar **Review and Launch**;
* Selecionar **Edit security groups** -> **Select an existing security group** -> escolher grupo criado previamente;
* Selecionar **Launch**;
* Será necessário então criar ou escolher um *key pair*, caso já tenha sido previamente criado;
* Selecionar o *key pair* e marcar a caixa de seleção;
* Selecionar **Launch instance**.

---
# Terceiro Passo - acesso à uma instância
---

Após alguns momentos de configuração a instância já deve estar inicializada e será possível acessá-la de duas maneiras diferentes:

## Acesso através do MindTerm - Terminal Java

* Acessar: https://console.aws.amazon.com/ec2/;
* Com o botão direito sobre a instância criada, selecionar **Connect**;
* Selecionar *A Java SSH Client directly from my browser (Java required)* - não funciona no Chrome devido à remoção de algum suporte ao Java. Rodei com o Firefox;
* O *User name* padrão da AMI da Amazon é *ec2-user*, basta mantê-lo;
* É necessário digitar/colar o path da máquina local onde se encontra a chave .pem previamente baixada;
* A opção *Save key location* pode ser marcada para evitar ter que procurar a localização em acessos futuros;
* Selecionar **Launch SSH Client** e autorizar o Java a rodar o MindTerm;
* No primeiro acesso o MindTerm vai querer alguns confirmações para uso;

O terminal da instância estará disponível através do MindTerm.

## Acesso através de SSH - Terminal local

Para acessar por SSH é necessário primeiramente configurar credenciais de segurança para o usuário do AWS - o subusuário previamente criado no IAM.

* Acessar: https://console.aws.amazon.com/iam/;
* Selecionar o usuário desejado e selecionar a aba *Security Credentials*;
* Selecionar **Create access key** e baixar o arquivo *.csv*, pois este é o único momento em que terá acesso a ele.

Após isto, será necessário instalar e configurar o AWS CLI (ferramenta de linha de comando do AWS):

* ``pip install awscli``
* ``aws configure``
* Preencher os campos *AWS Access Key ID* e *AWS Secret Access Key* com os valores providos no arquivo *.csv*;
* Preencher a região: us-west-2 (para Oregon, que está atualmente sendo utilizado)
* Deixar o último campo em branco e finalizar.

Por fim, para conectar-se à instância, que já deve estar rodando, deve-se digitar o seguinte comando:

* ``ssh -i "<path_arquivo_chave_pem>" ec2-user@<dns_público_instância>``

---
# Quarto Passo - instalação do Docker
---

Tutorial: http://docs.aws.amazon.com/AmazonECS/latest/developerguide/docker-basics.html

No MindTerm digitar as seguintes linhas:

* ``sudo yum update -y``
* ``sudo yum install -y docker``
* ``sudo service docker start``
* ``sudo usermod -a -G docker ec2-user``

Talvez seja necessário reiniciar o sistema para que as permissões sejam atualizadas:
* ``sudo reboot``
* *Enter* quando aparecer o dns da máquina para reconectar ao MindTerm;
* *Enter* quando aparecer o login da máquina (``ec2-user``)
* ``docker info`` - mostrará se o docker está rodando

---
# Quinto Passo - rodando o Jupyter através do Docker
---

Tutorial: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/README.md

Docker do curso de Deep Learning do Udacity:

* ``docker run -p 8888:8888 --name tensorflow-udacity -it gcr.io/tensorflow/udacity-assignments:0.6.0``

Onde 8888 é a porta configurada pelo Jupyter dentro do Docker, e também a porta configurada previamente com o grupo de segurança, para ser acessada de fora da máquina.

---
# Sexto Passo - acesso ao Jupyter
---

O endereço a ser acessado é o endereço que está em Public DNS na linha da instância em execução, na página do console EC2.

No formato ``<address>:8888`` ou a devida porta utilizada

---
# Informações adicionais
---

## Informações gerais

* Os dados colocados nas instâncias EC2 são persistidos sem problemas no armazenamento EBS.
* Enquanto o container executado no Docker existe, os dados são persistidos, mesmo que sua execução seja parada (``docker stop <nome_instancia_container>``) ou a instância da Amazon seja desligada (neste caso o container também é parado, então ao ligar a instância é necessário iniciá-lo, da mesma forma (``docker start <nome_instancia_docker>``).
* De qualquer forma, parece mais interessante garantir que os dados sejam persistidos fora do Docker, caso ocorra algum problema e o container precise ser instanciado novamente.
* Se possível, seria interessante inicializar a instância do AWS e salvar os dados (baixar do repositório, por exemplo) a serem utilizados no curso antes de conectar-se ao Docker, pois a partir daí o mesmo poderia ser iniciado uma única vez, já montando a pasta na instância do AWS dentro da instância do Docker.
* Em todo caso, se o material já vier na imagem do Docker, basta inicializar uma instância do Docker, copiar o conteúdo para a instância do AWS (``docker cp <nome_instancia_container>:<diretório_docker> <diretório_aws>``).


## Informações sobre o serviço ECS (EC2 Container Service)

* Uma maneira de automatizar um pouco a execução de instâncias do Docker.
* Não dá a opção de usar o tipo de instância ``g2.2xlarge``, que usa GPU.
* É necessário criar um cluster, um serviço e uma tarefa, mas isto não é tão complicado.
* Parece que ao matar uma instância o serviço automaticamente cria outra no lugar, então foi um pouco chato de remover as instâncias.
* Precisaria ver a questão de copiar os dados para a instância AWS para que ao inicializar a instância do Docker a mesma já pudesse montar os diretórios do curso corretamente.